In [181]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
data = pd.read_excel('C:/Users/Анна/Downloads/online_retail.xlsx')
data.to_csv('online_retail.csv', index=False)
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkFirst") \
      .getOrCreate() 
df = spark.read.csv('online_retail.csv', header=True)
#df.show(5)

In [182]:
def amn(Quantity,UnitPrice):
    return Quantity*UnitPrice
    
#df = df.withColumn('Amount', amn(df['Quantity'], df['UnitPrice']))

df = df.withColumn('InvoiceDate', date_format(df['InvoiceDate'], 'yyyy-MM-dd'))
df = df.withColumn('InvoiceDate', to_date(df['InvoiceDate'], 'yyyy-MM-dd'))
recency_df = df.groupBy('CustomerID').agg(max('InvoiceDate').alias('last_purchase_date'))
recency_df = recency_df.withColumn('recency', datediff(current_date(), recency_df['last_purchase_date']))
frequency_df = df.groupBy('CustomerID').agg(count('InvoiceNo').alias('frequency'))

monetary_df = df.groupBy('CustomerID').agg(round(sum(amn(df['Quantity'], df['UnitPrice'])), 2).alias('monetary_value'))

df = df.join(recency_df.select('CustomerID', 'recency'), 'CustomerID')
df = df.join(frequency_df.select('CustomerID', 'frequency'), 'CustomerID')
df = df.join(monetary_df.select('CustomerID', 'monetary_value'), 'CustomerID')
#df.show(3)

In [183]:
RFM_data=df['CustomerID', 'recency', 'frequency', 'monetary_value']
RFM_data.show(3)

+----------+-------+---------+--------------+
|CustomerID|recency|frequency|monetary_value|
+----------+-------+---------+--------------+
|   15039.0|   4286|     1508|      19786.44|
|   15039.0|   4286|     1508|      19786.44|
|   15039.0|   4286|     1508|      19786.44|
+----------+-------+---------+--------------+
only showing top 3 rows



In [184]:
import numpy as np

quantiles_recency = df.approxQuantile('recency', [0.33, 0.66], 0.01) 
df = df.withColumn('recency_group',
                   when(df['recency'] <= quantiles_recency[0], 'C')
                   .when((df['recency'] > quantiles_recency[0]) & (df['recency'] <= quantiles_recency[1]), 'B')
                   .otherwise('A'))

quantiles_frequency = df.approxQuantile('frequency', [0.33, 0.66], 0.01)
df = df.withColumn('frequency_group',
                   when(df['frequency'] <= quantiles_frequency[0], 'C')
                   .when((df['frequency'] > quantiles_frequency[0]) & (df['frequency'] <= quantiles_frequency[1]), 'B')
                   .otherwise('A'))

quantiles_monetary = df.approxQuantile('monetary_value', [0.33, 0.66], 0.01)
df = df.withColumn('monetary_group',
                   when(df['monetary_value'] <= quantiles_monetary[0], 'C')
                   .when((df['monetary_value'] > quantiles_monetary[0]) & (df['monetary_value'] <= quantiles_monetary[1]), 'B')
                   .otherwise('A'))

In [186]:
df = df.withColumn('RFM', concat_ws('', df['recency_group'], df['frequency_group'], df['monetary_group']))
df=df.filter(df["RFM"] == "AAA").select("CustomerID").distinct()
df.toPandas().to_csv('result.csv', header=False, index=False)
df.show()

+----------+
|CustomerID|
+----------+
|   17865.0|
|   17509.0|
|   12472.0|
|   15719.0|
|   16843.0|
|   13209.0|
|   13097.0|
|   12484.0|
|   14800.0|
|   12637.0|
|   16729.0|
|   17107.0|
|   16745.0|
|   15249.0|
|   16984.0|
+----------+

